In [ ]:
import json
import requests
import pandas as pd
import datetime
import time
import janitor

from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity="all"

# Set API urls
url_all_country = 'https://api.covid19api.com/countries'
summary_url = 'https://api.covid19api.com/summary'

def request_url(urls, return_response = False):
    '''
    Function to make request given an url.
    
    INPUT:
        urls - String of a given API address. 
    OUTPUT:
        r - String of response in Json  format.
    '''
    try:
        r = requests.get(url=urls)
        if return_response:
            return r
        return r.json()
    except Exception as e:
        print(e)
        print(f"Houve um erro ao tentar fazer a requisicao da url: {urls}")


def request_data_by_country(country_name, cases = ["confirmed","deaths"]):
    '''
    Function to make request and get response for more than one country in a given standard API URL
    since 2020-01-01.
    
    INPUT:
        country_name - List of String containing country Slugs.
        cases(Optional) - List of Strings containing type of update "confirmed","deaths" or "recovery".
    OUTPUT:
        json_list - String of response in Json  format.
        country_request_err - List of String with Slugs that did not reach an API response or returned an error.
        
    '''
    today = datetime.datetime.utcnow().strftime('%Y-%m-%dT00:00:00Z')
    json_list = []
    country_request_reprocess = []
    
    for name in country_name:
        for case_type in cases:
            try:
                print(f'pais: {name} e tipo de caso:{case_type}')
                counter = 0
                urls = f'https://api.covid19api.com/country/{name}/status/{case_type}?from=2020-01-01T00:00:00Z&to={today}'
                response = request_url(urls, True)
                
                # Try to make the request 5 times if you don't get a response the first time
                while response.status_code != 200:
                    counter += 1
                    response = request_url(urls, True)
                    time.sleep(30)
                    if counter == 5:
                        break
                
                if len(response) > 0:
                    json_list.append(response.json())
                else:
                    country_request_reprocess.append(name)
                
                # timer to avoid request limit  
                time.sleep(6*60)
            except Exception as e:
                country_request_reprocess.append(name)
                print(e)
                print(f"Houve um erro ao tentar fazer a requisicao da url com o pais: {name} e tipo de caso:{case_type}")
        
    return json.dumps(json_list), country_request_reprocess

In [ ]:
def get_top_deaths_confirmed_contries(summary_dataframe, country_dataframe, n = 30):
    '''
    Function get top countries in deaths and confirmed cases.
    
    INPUT:
        summary_dataframe - Dataframe cointaing a summary of all COVID-19 cases.
        country_dataframe - Dataframe cointaing name of country, slug and ISO2 code.
        n (optional) - Integer number refered to top countries to return.
    OUTPUT:
        top_countries - List of strings containg top countries in both categories.
    '''
    
    # top rank Countries by confirmed cases
    top_TotalConfirmed = summary_dataframe.sort_values(by=['TotalConfirmed'], ascending = False).head(n)
    list_top_confirmed = list(top_TotalConfirmed['CountryCode'].unique())

    # top rank Countries by death
    top_deaths = summary_dataframe.sort_values(by=['TotalDeaths'], ascending = False).head(n)
    list_top_deaths = list(top_deaths['CountryCode'].unique())

    # List with tops Death and Confirmed cases
    countries_ISO = list(set(list_top_confirmed+list_top_deaths))

    # get name of countries in Slug
    top_countries = list(country_dataframe[country_dataframe['ISO2'].isin(countries_ISO)]['Slug'].unique())
    
    return top_countries

In [ ]:
def build_dataframe(json_agg):
    '''
    Function to consolidate all countries in json into a single dataframe.
    
    INPUT:
        json_agg - list of Json containg data from all top countries.   
    OUTPUT:
        dataframe_top_contries - String of response in Json  format.
    '''
    # iterate thru json and get all data 
    dataframe_top_contries = pd.DataFrame()
    for element in json_agg:
        dataframe_top_contries = dataframe_top_contries.append(pd.json_normalize(element), ignore_index=True)
    
    return dataframe_top_contries

In [ ]:
import psycopg2
class Postgres():
  def __init__(self,database='postgres',user='DataWarriorsAdmin@data-warriors-postgresql',password='DataWarriorsPassword!',host='data-warriors-postgresql.postgres.database.azure.com',port='5432'):
    self.connection = psycopg2.connect(host=host, database=database,user=user, password=password,port=port, sslmode='require')
    self.new_cursor()

  def get_connection(self):
    return self.connection

  def new_cursor(self):
    self.cursor = self.connection.cursor()
    return self.cursor

  def commit(self):
    self.connection.commit()

  def rollback(self):
    self.connection.rollback()

  def fetchall(self):
    return self.cursor.fetchall()

  def execute(self,query):
    self.cursor.execute(query)
    self.commit()

  def search(self,query):
    self.new_cursor()
    self.cursor.execute(query)
    self.commit()
    return self.fetchall()

  def insertMany(self,query,data_list):
    try:
      self.new_cursor()
      self.cursor.executemany(query, data_list)
      self.commit()    
      print('Execução da transação conclída com sucesso:', query)
    except Exception as error:
      self.rollback()
      print('Transação falhou:',error)
    
  def closeConnection(self):
    self.connection.close()


In [ ]:
import json
class ParseDFToDatabase(object):
  def __init__(self,df,table_name):
    self.df = df
    self.table_name = table_name
    self.tuple_list = self.dfToTupleList()
    self.columns = tuple(df.columns.to_list())
    self.format_colums = str(tuple([''.join('%s') for column in self.columns])).replace("'","")
    self.columns = str(tuple(self.columns)).replace("'","").upper()
    self.insert_query = f'INSERT INTO {self.table_name}{self.columns} VALUES{self.format_colums}'
  
  def dfToTupleList(self):
    df_json = json.loads(self.df.to_json(orient='records'))
    return list(map(lambda row: tuple(row.values()),df_json ))

  def get(self):
    return self.insert_query, self.tuple_list

# request countries
data_country = pd.json_normalize(request_url(url_all_country))

# request summary
data_sum = request_url(summary_url)
top = pd.json_normalize(data_sum['Countries'])

# get a list containing top 30 countries with confirmed and death cases
list_top_countries = get_top_deaths_confirmed_contries(top,data_country)

# make requestget and aggregate all top 30 countries in a single json
aggregate, list_to_reprocess = request_data_by_country(list_top_countries)
aggregate_json = json.loads(aggregate)

# save log file
today = datetime.datetime.utcnow().strftime('%Y-%m-%dT00:00:00Z')
with open(f'error_logs_{today}.txt', 'a') as writer:
    writer.writelines(f'Countries that request failed: {str(list_to_reprocess)} | Process date: {today}')

# make a datafrane of json file
df_top_30_by_cases = build_dataframe(aggregate_json)

In [ ]:
def preparaInsercaoDaily(df):
    df.drop(columns=['Country', 'Province', 
                 'City', 'CityCode', 'Lat', 'Lon',"message"], inplace=True)
    df["TOTALDEATHS"]=df["Cases"][df["Status"]=="deaths"]
    df["TOTALDEATHS"]=df["Cases"][df["Status"]=="deaths"]
    df["TOTALCONFIRMED"]=df["Cases"][df["Status"]=="confirmed"]
    df.fillna(0, inplace=True)
    df.drop(columns=["Cases","Status"], inplace=True)
    df=df.groupby(["Date","CountryCode"])[["TOTALDEATHS","TOTALCONFIRMED"]].sum()
    df=df.reset_index()
    df=df.rename({"Date":"DATEREG",
          "CountryCode":"COUNTRYCODE"},
         axis=1)
    df["DATEREG"]=pd.to_datetime(df['DATEREG']).dt.date
    df["DATEREG"]=df["DATEREG"].astype(str)
    grouped=df.groupby('COUNTRYCODE')
    df_novo=pd.DataFrame()
    for name, country in grouped:
        country["NEWCONFIRMED"]=country["TOTALCONFIRMED"].diff(1)
        country["NEWDEATHS"]=country["TOTALDEATHS"].diff(1)
        df_novo=pd.concat([df_novo,country], ignore_index=True)
    df=df_novo.fillna(0)
    df=df.astype({"TOTALCONFIRMED":int, "TOTALDEATHS":int, "NEWCONFIRMED":int, "NEWDEATHS":int})
    df=df[['DATEREG', 
       'COUNTRYCODE', 
       'TOTALCONFIRMED',
       'TOTALDEATHS', 
       'NEWCONFIRMED', 
       'NEWDEATHS']]
    return df

In [ ]:
def preparaInsercaoSummary():
    grouped=df.groupby('COUNTRYCODE')
    df_summary=pd.DataFrame()
    for name, country in grouped:
        reg=country.tail(1)
        df_summary=pd.concat([df_summary,reg], ignore_index=True)
    df_summary=df_summary.rename({"DATEREG":"LASTUPDATED"}, 
                    axis=1)
    
    return df_summary

In [ ]:
def fazInsercao(df,tabela,banco):
    x=ParseDFToDatabase(df,tabela)
    query=x.get()[0]
    dados=x.get()[1]
    banco.insertMany(query,dados)


### Parte guto

In [ ]:
# request countries
data_country = pd.json_normalize(request_url(url_all_country))

# request summary
data_sum = request_url(summary_url)
top = pd.json_normalize(data_sum['Countries'])

# get a list containing top 30 countries with confirmed and death cases
list_top_countries = get_top_deaths_confirmed_contries(top,data_country)

# make requestget and aggregate all top 30 countries in a single json
aggregate, list_to_reprocess = request_data_by_country(list_top_countries)
aggregate_json = json.loads(aggregate)

# make a datafrane of json file
df_top_30_by_cases = build_dataframe(aggregate_json)
df = f_top_30_by_cases.dropna(subset = ['Country'])

### Parte Carlos

In [ ]:
# Prepara o dataframe gerado pelo acesso à api para inserção na base
#
df=preparaInsercaoDaily(df)

# Prepara o dataframe df_summary para inserção na base
df_summary=preparaInsercaoSummary(df)

#Connecta no banco
banco=Postgres()
conn=banco.get_connection()
cursor=banco.new_cursor()

fazInsercao(df,"DAILY",banco)
fazInsercao(df_summary,"SUMMARY",banco)


In [ ]:
SELECT datereg from public.daily 
order by datereg desc 
limit 1;